# Data Cleaning Task
#### 2/24/23
https://docs.google.com/document/d/1JbR3mLBW1kZCSrqBYMhQs4AC-4Hk5OOO2UCLflP76eY/edit

## Understanding the dataset

In [1]:
import numpy as np # Used once for np.nan
import pandas as pd
from scipy.stats import zscore
import datetime # Used to get datetime from unix timestamp

pd.options.mode.chained_assignment = None # Avoids an unnecessary warning

In [2]:
df = pd.read_csv("results_0221.csv") # Import the original results dataset
df.shape

(15150, 26)

### Quick notes and observations

(I'm looking at the dataset on Numbers, since jupyterhub can't easily display the whole table.) 

The original dataset has 15,150 rows and 26 columns. It includes all the data received from the experiment divided into separate user entries. Each entry has information on the answers to the consent survey, the experiment (as well as the practice trials), and the background survey.

Maho pointed out that it seems like some users participated more than once. Since the desired effect might be different on familiar users, it's important to identify these users and remove their repeated entries.
 
Column info:
1. The time the results were received (Unix timestamp) - all of these will be the same for each row in a given entry.
2. IP address hash. Likewise, all of these will be the same for each row in a given entry. They will also be the same for a given user, so it can be used to find the users who participated more than once.
3. Only says "PennController", so not useful.  
4. The items' order number. These are all grouped in chunks of 5. Not sure what the "item" refers to.
5. Only says 0, so not useful.
6. The type of question (consent, exercise, experiment-filler, experiment-critical, or background)
7. Only says NULL, so not useful.
8. The type of element displayed ("Scale" occurs every 5 rows).
9. The name given to the above.
10. "Parameter" - unclear to me.
11. Value - seems to include the choice that the user picks on the scale (Important!).
12. "Event" time - probably not useful.
13. Only says undefined, so not useful.

Columns 14-21 include answers to the background surveys.

22. The list that the stimulus was taken from (always the same for a given participant's experiment trials).
23. The stimulus sentence's order number.
24. The condition (either practice, good, intermediate, bad, or one of the 4 critical conditions).
25. The actual sentence.
26. The comments (seem to be mostly empty, but worth checking before deleting).

## Step 1: Extract submission time

Instead of extracting the submission time first, I'll extract it later during Step 6 by converting the Unix timestamps into date times.

## Step 2: Extract relevant rows

In [3]:
# Only include rows that say "experiment-filler", "experiment-critical", or "background" under column V6.
df = df[(df["V6"] == "experiment-filler") | 
        (df["V6"] == "experiment-critical") | 
        (df["V6"] == "background")]
df.shape

(12337, 26)

In [4]:
# Test: Are there only three trial types in V6?
assert df["V6"].nunique() == 3

## Step 3: Identify and recode unique subject IDs

In [5]:
# Change column V1 from Unix time to an ordinal number, starting at 1.
df["V2"] = df.groupby("V2", sort=False).ngroup() + 1
df = df.rename(columns={"V2": "subj_id"})

In [6]:
# Test: is the list of users equal to the numbers 1...35?
assert df["subj_id"].drop_duplicates().to_list() == list(range(1, 36))

## Step 4: Find and remove multiple submissions from the same user

In [7]:
n_submissions = df.groupby("subj_id")["V1"].nunique() # Find the number of submissions per user.
n_submissions.value_counts()

1    28
2     7
Name: V1, dtype: int64

There were 35 users total: 28 submitted once, 7 submitted twice, and no one submitted more than twice.

In order to remove the duplicate entries from the dataset, my strategy is:
1. Remove ALL duplicates in the subj_id column.
2. Remember which timestamps designate a user's first entry.
3. Remove all rows where the timestamp does not designate a user's first entry.

In [8]:
# List of timestamps that designate a user's first entry.
first_entry_ts = df.drop_duplicates(subset="subj_id", keep="first")["V1"].to_list()
# Test: since there are 35 users, there should be 35 first entries.
assert len(first_entry_ts) == 35 

In [9]:
df = df[df["V1"].isin(first_entry_ts)]
n_submissions = df.groupby("subj_id")["V1"].nunique()
n_submissions.value_counts()

1    35
Name: V1, dtype: int64

Now, the dataset does not include any repeated submissions – all 35 users only have one each.

## Step 5: Separate acceptability judgments and responses to language background questionnaire

In [10]:
df_ju = df[(df["V6"] == "experiment-filler") | 
             (df["V6"] == "experiment-critical")]
df_bg = df[df["V6"] == "background"]

In [11]:
# Test: in the judgment dataset, are there only two trial types in V6?
assert df_ju["V6"].nunique() == 2

In [12]:
# Test: in the background dataset, is there only one trial type in V6?
assert df_bg["V6"].nunique() == 1

## Step 6: Extract relevant information from the subject info dataset

As I mentioned in Step 1, I'll extract the submission time now: 

In [13]:
timestamps = df_bg["V1"].drop_duplicates().reset_index(drop=True)
submission_times = timestamps.apply(lambda s: datetime.datetime.utcfromtimestamp(int(s)))
submission_times.head()

0   2023-02-14 17:08:33
1   2023-02-14 17:43:31
2   2023-02-14 18:52:07
3   2023-02-14 18:56:30
4   2023-02-14 21:13:03
Name: V1, dtype: datetime64[ns]

In [14]:
df_bg = df_bg[(df_bg["V10"] == "Final") | (df_bg["V10"] == "Choice")]
df_bg = df_bg[["subj_id", "V9", "V11"]]
df_bg = pd.pivot(df_bg, index="subj_id", columns="V9", values="V11").reset_index()
df_bg.head()

V9,subj_id,input_US_age,input_age,input_birth,input_comf,input_gender,input_lang,input_lang_comf,input_parent
0,1,NaN,20,In the U.S.,English,Female,NaN,NaN,English
1,2,NaN,22,In the U.S.,English,Female,NaN,NaN,English
2,3,NaN,22,In the U.S.,English,Female,NaN,NaN,English
3,4,NaN,20,In the U.S.,English,Female,NaN,NaN,English
4,5,NaN,20,In the U.S.,English,Female,Vietnamese,NaN,Others


The index column on the left titled "V9" will be removed when the dataset is exported.

Other than that, there's a minor formatting issue where commas are represented as "%2C". I'll fix that real quick.

In [15]:
# If the input is a string, replace "%2C" with a comma.
def comma_if_str(s):
    if type(s) == str:
        return s.replace("%2C", ",")
    return np.nan

df_bg["input_lang"] = df_bg["input_lang"].apply(comma_if_str)
df_bg[31:34]

V9,subj_id,input_US_age,input_age,input_birth,input_comf,input_gender,input_lang,input_lang_comf,input_parent
31,32,NaN,21,In the U.S.,English,Female,"First English, then a bit of Tagalog (Filipino)",NaN,Others
32,33,NaN,19,In the U.S.,English,Male,"Japanese, English",NaN,Others
33,34,NaN,19,In the U.S.,English,Female,"Kannada, Tamil",NaN,Others


## Step 7: Add the submission date extracted in Step 1 to the subject info dataset

In [16]:
df_bg["submission_time"] = submission_times
df_bg.head()

V9,subj_id,input_US_age,input_age,input_birth,input_comf,input_gender,input_lang,input_lang_comf,input_parent,submission_time
0,1,NaN,20,In the U.S.,English,Female,NaN,NaN,English,2023-02-14 17:08:33
1,2,NaN,22,In the U.S.,English,Female,NaN,NaN,English,2023-02-14 17:43:31
2,3,NaN,22,In the U.S.,English,Female,NaN,NaN,English,2023-02-14 18:52:07
3,4,NaN,20,In the U.S.,English,Female,NaN,NaN,English,2023-02-14 18:56:30
4,5,NaN,20,In the U.S.,English,Female,Vietnamese,NaN,Others,2023-02-14 21:13:03


The background dataset is finished and is stored as __df_bg__.

## Step 8: Extract relevant information from the acceptability judgment data

In [17]:
df_ju = df_ju[(df_ju["V10"] == "Choice") | (df_ju["V10"] == "Selection")]
df_ju = df_ju[["subj_id", "V22", "V23", "V6", "V24", "V25", "V11"]]
df_ju.head()

,subj_id,V22,V23,V6,V24,V25,V11
68,1,1,9.0,experiment-filler,bad,I had every hour many days swum 100 meters in ...,2
73,1,1,8.0,experiment-filler,good,Everyone takes advantage of the few sunny days...,6
78,1,1,7.0,experiment-filler,intermediate,Do you remember the girl who Jack commented on...,5
83,1,1,11.0,experiment-critical,nested_island,I talked about the man that I like the truck t...,3
88,1,1,18.0,experiment-filler,bad,Who edited the article that we will have acces...,5


In [18]:
df_ju.columns = ["subj_id", "list", "item", "trial_type", "condition", "sentence", "judgment"]
df_ju.head()

,subj_id,list,item,trial_type,condition,sentence,judgment
68,1,1,9.0,experiment-filler,bad,I had every hour many days swum 100 meters in ...,2
73,1,1,8.0,experiment-filler,good,Everyone takes advantage of the few sunny days...,6
78,1,1,7.0,experiment-filler,intermediate,Do you remember the girl who Jack commented on...,5
83,1,1,11.0,experiment-critical,nested_island,I talked about the man that I like the truck t...,3
88,1,1,18.0,experiment-filler,bad,Who edited the article that we will have acces...,5


In [19]:
# Quickly gonna make the item column int instead of float.
df_ju["item"] = df_ju["item"].apply(int)

## Step 9: Convert acceptability judgments to z-scores

In [20]:
# The ints in the judgment column are actually strings – need to typecast them first.
df_ju["judgment"] = df_ju["judgment"].apply(int)

In [21]:
# Apply corresponding zscores.
df_ju["zscore"] = df_ju.groupby("subj_id")["judgment"].apply(lambda x: zscore(x, ddof=1))
df_ju.head()

,subj_id,list,item,trial_type,condition,sentence,judgment,zscore
68,1,1,9,experiment-filler,bad,I had every hour many days swum 100 meters in ...,2,-1.000603
73,1,1,8,experiment-filler,good,Everyone takes advantage of the few sunny days...,6,1.094118
78,1,1,7,experiment-filler,intermediate,Do you remember the girl who Jack commented on...,5,0.570437
83,1,1,11,experiment-critical,nested_island,I talked about the man that I like the truck t...,3,-0.476923
88,1,1,18,experiment-filler,bad,Who edited the article that we will have acces...,5,0.570437


Quick note – I tried to calculate z-score using mean and stdev, and found that two of the results had a __stdev of 0__ (specifically subj 17 and 22):

In [22]:
stds = df_ju.groupby("subj_id")["judgment"].std()
stds[stds == 0]

subj_id
17    0.0
22    0.0
Name: judgment, dtype: float64

Which means that subj 17 and 22 answered the same for every trial. Maho says we should remove them (and others who might not have been paying attention) from the dataset.

## Attention check

Procedure:

1. Get a list of 10 fillers whose mean acceptability was the highest, and 10 lowest (20 in total).
2. Check each participant's judgment of each of the 20 fillers, and see if their judgment was more than 2 standard deviations away from the mean
3. If a participant's judgment met this criterion more than 5 out of 20 times, exclude the participant from data analysis 

### Part 1

In [23]:
# Only include filler sentences
df_check = df_ju[df_ju["trial_type"] == "experiment-filler"]
df_check.head()

,subj_id,list,item,trial_type,condition,sentence,judgment,zscore
68,1,1,9,experiment-filler,bad,I had every hour many days swum 100 meters in ...,2,-1.000603
73,1,1,8,experiment-filler,good,Everyone takes advantage of the few sunny days...,6,1.094118
78,1,1,7,experiment-filler,intermediate,Do you remember the girl who Jack commented on...,5,0.570437
88,1,1,18,experiment-filler,bad,Who edited the article that we will have acces...,5,0.570437
93,1,1,9,experiment-filler,intermediate,Which teachers are the administrator firing at...,6,1.094118


In [24]:
# Find the overall mean score for each sentence
sent_means = df_check.groupby("sentence")["judgment"].mean()
df_check["sent_mean"] = sent_means.loc[df_check["sentence"]].values

# Find the overall z-score for each judgment
df_check["sent_zscore"] = df_check.groupby("sentence")["judgment"].apply(lambda x: zscore(x, ddof=1))
df_check.head()

,subj_id,list,item,trial_type,condition,sentence,judgment,zscore,sent_mean,sent_zscore
68,1,1,9,experiment-filler,bad,I had every hour many days swum 100 meters in ...,2,-1.000603,1.885714,0.111778
73,1,1,8,experiment-filler,good,Everyone takes advantage of the few sunny days...,6,1.094118,5.885714,0.075676
78,1,1,7,experiment-filler,intermediate,Do you remember the girl who Jack commented on...,5,0.570437,4.942857,0.036380
88,1,1,18,experiment-filler,bad,Who edited the article that we will have acces...,5,0.570437,3.400000,1.162563
93,1,1,9,experiment-filler,intermediate,Which teachers are the administrator firing at...,6,1.094118,5.000000,0.550973


In [25]:
# Find the 10 best and 10 worst sentences (according to the sent_mean column)
df_sort = df_check[["sentence", "sent_mean"]].drop_duplicates("sentence").sort_values(by="sent_mean")
df_sort = pd.concat((df_sort.head(10), df_sort.tail(10)))
df_sort

,sentence,sent_mean
68,I had every hour many days swum 100 meters in ...,1.885714
303,The famous siblings together became we 17 peop...,2.000000
283,To he likes that linguistics would like to the...,2.028571
248,Peeled the banana in the kitchen the waitress ...,2.142857
263,There was love one’s true self many men now fr...,2.200000
228,What will you buys when you could went on a va...,2.314286
198,The students described which counselor the dep...,2.600000
233,What did Sarah eat that Bob likes a pizza more...,2.657143
338,What did the talented chef cook ravioli with s...,2.771429
113,The lawyer the judge determined the defendant ...,2.800000


### Part 2

In [26]:
# Only include sentences that are in the above dataset ^
df_check = df_check[df_check["sentence"].isin(df_sort["sentence"])]

# Only include judgments that were significantly different from the mean
df_check = df_check[abs(df_check["sent_zscore"]) > 2]
df_check

,subj_id,list,item,trial_type,condition,sentence,judgment,zscore,sent_mean,sent_zscore
263,1,1,11,experiment-filler,bad,There was love one’s true self many men now fr...,6,1.094118,2.200000,2.694944
313,1,1,10,experiment-filler,good,The city closed your favorite pizzeria due to ...,2,-1.000603,6.342857,-3.272302
948,3,2,9,experiment-filler,bad,I had every hour many days swum 100 meters in ...,4,0.457816,1.885714,2.067888
1213,4,1,3,experiment-filler,good,Nobody liked the old garden because it was unk...,2,-1.029646,6.200000,-3.023687
1223,4,1,17,experiment-filler,bad,What did Sarah eat that Bob likes a pizza more...,6,1.276761,2.657143,2.055947
1243,4,1,6,experiment-filler,good,Somebody mentioned that you liked wine so I br...,2,-1.029646,6.057143,-2.614302
1358,4,1,1,experiment-filler,bad,To he likes that linguistics would like to the...,6,1.276761,2.028571,2.351605
1378,4,1,12,experiment-filler,bad,The famous siblings together became we 17 peop...,6,1.276761,2.000000,2.748737
1598,5,2,7,experiment-filler,good,There was an enormous painting of a Teddy bear...,1,-1.096681,5.857143,-3.242640
1889,6,3,15,experiment-filler,bad,The students described which counselor the dep...,7,1.588985,2.600000,2.974443


### Part 3

In [27]:
# Find which subject(s) to remove
should_remove = (df_check["subj_id"].value_counts() >= 5)
remove_subj = should_remove.index[should_remove]
remove_subj.to_list()

[6, 4, 28]

In [28]:
# Remove these subjects
df_ju = df_ju[~df_ju["subj_id"].isin(remove_subj)]
df_bg = df_bg[~df_bg["subj_id"].isin(remove_subj)]

### Notes

Looking at the judgments from subject 4, 6, and 28, it seems like this attention check did a pretty good job of finding subjects who might not have been paying attention. However, it misses subject 17 and 22 (the two subjects who answered "4" every time). This is because there is only one filler sentence for which a judgment of 4 is significantly different from the mean:

In [29]:
df_check[df_check["judgment"] == 4]

,subj_id,list,item,trial_type,condition,sentence,judgment,zscore,sent_mean,sent_zscore
948,3,2,9,experiment-filler,bad,I had every hour many days swum 100 meters in ...,4,0.457816,1.885714,2.067888
6605,17,1,9,experiment-filler,bad,I had every hour many days swum 100 meters in ...,4,NaN,1.885714,2.067888
9607,22,1,9,experiment-filler,bad,I had every hour many days swum 100 meters in ...,4,NaN,1.885714,2.067888


Is it worth adding another kind of attention check in order to remove subjects like 17 and 22 from the dataset? Obviously, I'll definitely want to make sure my code is correct first, so let me know if you see any errors or get different results (or if you have a more efficient/organized process).

## Step 10: Export the dataset

In [30]:
# index=False: the ugly index column on the left will go away.
df_bg.to_csv("background_data_clean.csv", index=False)
df_ju.to_csv("judgment_data_clean.csv", index=False)

## Bonus: Find a better way to clean the dataset

Maho pointed out that the index column at the start of each entry could be used to find the corresponding columns. That way, we wouldn't need to manually figure out which column is which, nor would we need to rename them.

My strategy is to collect these index columns, clean the dataset the same way as before up to Step 6, and add the index columns back as the new headers before extracting relevant columns. Then continue cleaning, but without renaming the headers.

In [31]:
df = pd.read_csv("results_0221.csv")

In [32]:
# The index columns all start with "#" and a number.
# Return true if a string fits the above description.
def is_index_col(s):
    if type(s) == str:
        if len(s) > 2:
            if "#" in s and s[2].isnumeric():
                return True
    return False

index_cols = list(df[df["V1"].apply(is_index_col)]["V1"].unique())
index_cols

['# 1. Results reception time.',
 "# 2. MD5 hash of participant's IP address.",
 '# 3. Controller name.',
 '# 4. Order number of item.',
 '# 5. Inner element number.',
 '# 6. Label.',
 '# 7. Latin Square Group.',
 '# 8. PennElementType.',
 '# 9. PennElementName.',
 '# 10. Parameter.',
 '# 11. Value.',
 '# 12. EventTime.',
 '# 13. id.',
 '# 14. lang_comf.',
 '# 15. comf.',
 '# 16. lang.',
 '# 17. parents.',
 '# 18. age_US.',
 '# 19. birth.',
 '# 20. gender.',
 '# 21. age.',
 '# 22. Comments.',
 '# 22. LIST.',
 '# 23. ITEM.',
 '# 24. CONDITION.',
 '# 25. SENTENCE.',
 '# 26. Comments.']

In [33]:
# For some reason, "Comments" and "LIST" are both labelled as #22. I'll remove "comments" manually.
index_cols.remove("# 22. Comments.")

There are 26 columns to match with the 26 "V" columns. Now continue cleaning the same way as before:

In [34]:
# Step 2
df = df[(df["V6"] == "experiment-filler") | 
        (df["V6"] == "experiment-critical") | 
        (df["V6"] == "background")]

In [35]:
# Step 3
df["V2"] = df.groupby("V2", sort=False).ngroup() + 1

In [36]:
# Step 4
first_entry_ts = df.drop_duplicates(subset="V2", keep="first")["V1"].to_list()
df = df[df["V1"].isin(first_entry_ts)]

In [37]:
# Step 5
df_ju = df[(df["V6"] == "experiment-filler") | 
             (df["V6"] == "experiment-critical")]
df_bg = df[df["V6"] == "background"]

In [38]:
# Replace columns
df_ju.columns = index_cols
df_bg.columns = index_cols

In [39]:
df_ju.head()

,# 1. Results reception time.,# 2. MD5 hash of participant's IP address.,# 3. Controller name.,# 4. Order number of item.,# 5. Inner element number.,# 6. Label.,# 7. Latin Square Group.,# 8. PennElementType.,# 9. PennElementName.,# 10. Parameter.,...,# 17. parents.,# 18. age_US.,# 19. birth.,# 20. gender.,# 21. age.,# 22. LIST.,# 23. ITEM.,# 24. CONDITION.,# 25. SENTENCE.,# 26. Comments.
65,1676394513,1,PennController,32.0,0.0,experiment-filler,NaN,PennController,33,_Trial_,...,undefined,undefined,undefined,undefined,undefined,1,9.0,bad,I had every hour many days swum 100 meters in ...,NaN
66,1676394513,1,PennController,32.0,0.0,experiment-filler,NaN,PennController,33,_Header_,...,undefined,undefined,undefined,undefined,undefined,1,9.0,bad,I had every hour many days swum 100 meters in ...,NaN
67,1676394513,1,PennController,32.0,0.0,experiment-filler,NaN,PennController,33,_Header_,...,undefined,undefined,undefined,undefined,undefined,1,9.0,bad,I had every hour many days swum 100 meters in ...,NaN
68,1676394513,1,PennController,32.0,0.0,experiment-filler,NaN,Scale,judgment,Choice,...,undefined,undefined,undefined,undefined,undefined,1,9.0,bad,I had every hour many days swum 100 meters in ...,NaN
69,1676394513,1,PennController,32.0,0.0,experiment-filler,NaN,PennController,33,_Trial_,...,undefined,undefined,undefined,undefined,undefined,1,9.0,bad,I had every hour many days swum 100 meters in ...,NaN


In [40]:
df_bg.head()

,# 1. Results reception time.,# 2. MD5 hash of participant's IP address.,# 3. Controller name.,# 4. Order number of item.,# 5. Inner element number.,# 6. Label.,# 7. Latin Square Group.,# 8. PennElementType.,# 9. PennElementName.,# 10. Parameter.,...,# 17. parents.,# 18. age_US.,# 19. birth.,# 20. gender.,# 21. age.,# 22. LIST.,# 23. ITEM.,# 24. CONDITION.,# 25. SENTENCE.,# 26. Comments.
346,1676394513,1,PennController,64.0,0.0,background,NaN,PennController,2,_Trial_,...,English,NaN,In the U.S.,Female,20,NaN,NaN,NaN,NaN,NaN
347,1676394513,1,PennController,64.0,0.0,background,NaN,PennController,2,_Header_,...,English,NaN,In the U.S.,Female,20,NaN,NaN,NaN,NaN,NaN
348,1676394513,1,PennController,64.0,0.0,background,NaN,PennController,2,_Header_,...,English,NaN,In the U.S.,Female,20,NaN,NaN,NaN,NaN,NaN
349,1676394513,1,PennController,64.0,0.0,background,NaN,TextInput,input_age,Final,...,English,NaN,In the U.S.,Female,20,NaN,NaN,NaN,NaN,NaN
350,1676394513,1,PennController,64.0,0.0,background,NaN,TextInput,input_age,First,...,English,NaN,In the U.S.,Female,20,2,NaN,NaN,NaN,NaN


Looks like it worked! All the columns seem to line up. It might have been better to rename V2 as "subj_id," but other than that, this seems to be helpful. Now continue cleaning as before:

In [41]:
# Continuing Step 6
c_time = "# 1. Results reception time."
timestamps = df_bg[c_time].drop_duplicates().reset_index(drop=True)
submission_times = timestamps.apply(lambda s: datetime.datetime.utcfromtimestamp(int(s)))

In [42]:
# Step 7
c_param = "# 10. Parameter."
c_subj = "# 2. MD5 hash of participant's IP address."
c_quest = "# 9. PennElementName."
c_val = "# 11. Value."

df_bg = df_bg[(df_bg[c_param] == "Final") | 
              (df_bg[c_param] == "Choice")]
df_bg = df_bg[[c_subj, c_quest, c_val]]

df_bg = pd.pivot(df_bg, index=c_subj, columns=c_quest, values=c_val).reset_index()
df_bg["input_lang"] = df_bg["input_lang"].apply(comma_if_str)
df_bg["submission_time"] = submission_times
df_bg.head()

# 9. PennElementName.,# 2. MD5 hash of participant's IP address.,input_US_age,input_age,input_birth,input_comf,input_gender,input_lang,input_lang_comf,input_parent,submission_time
0,1,NaN,20,In the U.S.,English,Female,NaN,NaN,English,2023-02-14 17:08:33
1,2,NaN,22,In the U.S.,English,Female,NaN,NaN,English,2023-02-14 17:43:31
2,3,NaN,22,In the U.S.,English,Female,NaN,NaN,English,2023-02-14 18:52:07
3,4,NaN,20,In the U.S.,English,Female,NaN,NaN,English,2023-02-14 18:56:30
4,5,NaN,20,In the U.S.,English,Female,Vietnamese,NaN,Others,2023-02-14 21:13:03


In [43]:
# Step 8
c_label = "# 6. Label."
c_list = "# 22. LIST."
c_item = "# 23. ITEM."
c_cond = "# 24. CONDITION."
c_sent = "# 25. SENTENCE."

df_ju = df_ju[(df_ju[c_param] == "Choice") | (df_ju[c_param] == "Selection")]
df_ju = df_ju[[c_subj, c_list, c_item, c_label, c_cond, c_sent, c_val]]
df_ju[c_item] = df_ju[c_item].apply(int)

In [44]:
# Step 9
df_ju[c_val] = df_ju[c_val].apply(int)
df_ju["zscore"] = df_ju.groupby(c_subj)[c_val].apply(lambda x: zscore(x, ddof=1))
df_ju.head()

,# 2. MD5 hash of participant's IP address.,# 22. LIST.,# 23. ITEM.,# 6. Label.,# 24. CONDITION.,# 25. SENTENCE.,# 11. Value.,zscore
68,1,1,9,experiment-filler,bad,I had every hour many days swum 100 meters in ...,2,-1.000603
73,1,1,8,experiment-filler,good,Everyone takes advantage of the few sunny days...,6,1.094118
78,1,1,7,experiment-filler,intermediate,Do you remember the girl who Jack commented on...,5,0.570437
83,1,1,11,experiment-critical,nested_island,I talked about the man that I like the truck t...,3,-0.476923
88,1,1,18,experiment-filler,bad,Who edited the article that we will have acces...,5,0.570437


In [45]:
# Step 10
#df_bg.to_csv("background_data_clean_bonus.csv", index=False)
#df_ju.to_csv("judgment_data_clean_bonus.csv", index=False)